In [4]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from util.ZigZag import ZigZag
from util.BuildData import BuildData2
from util.CalcReturns import CalcReturns

from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [9]:
#df = pd.read_csv('EURUSD1.csv', columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
df = pd.read_csv('EURUSD1.csv', header=0)
train = np.array(df.Close)

In [12]:
Z = ZigZag(train, 0.0004)
returns = CalcReturns(train)

S = BuildData2(Z, returns, 61)

Dim = S['data'].shape[1]
t_count = 256

#D, L = BuildData1(Z, returns, 61)
#D, L = BuildData0(Z, train, 61)

print(S['data'].shape)
print(len(S['data']))


(1414, 60)
1414


In [13]:
#clf = svm.SVC(kernel="poly", C=0.0001, degree=6, coef0=0.0001)
#clf = RandomForestClassifier(n_estimators = 100, criterion='entropy') # 79%
#clf = ExtraTreesClassifier(n_estimators = 10, oob_score = True, bootstrap = True)
#clf = ExtraTreesClassifier(n_estimators = 100)
#clf = DecisionTreeClassifier()
#clf = AdaBoostClassifier(learning_rate = 0.6, n_estimators = 100) # 71%
#clf = GradientBoostingClassifier(n_estimators = 100, learning_rate=1.1) # 81%
#clf = BaggingClassifier() #79 %
#clf = KNeighborsClassifier(n_neighbors=100, algorithm='brute') # 37%
#clf = BaggingClassifier(base_estimator = KNeighborsClassifier(n_neighbors=100)) # 36%
clf = BaggingClassifier(base_estimator = GradientBoostingClassifier(n_estimators = 100, learning_rate=0.1)) # 81%

#clf0 = GradientBoostingClassifier(n_estimators = 100, learning_rate=0.1) # 81%
#clf1 = KNeighborsClassifier(n_neighbors=100, algorithm='brute') # 37%

#clf = VotingClassifier(estimators=[('gbc', clf0),('knc', clf1)], voting='soft')
clf.fit(S['data'][:-t_count], S['label'][:-t_count])

BaggingClassifier(base_estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [14]:
correct_count  = 0.0

test_buy  = 1.0
test_sell = 1.0
test_hold = 1.0

correct_buy  = 0.0
correct_sell = 0.0
correct_hold = 0.0

incorrect_buy  = 0.0
incorrect_sell = 0.0
incorrect_hold = 0.0

p = clf.predict(S['data'][len(S['data'])-t_count:])
c = S['label'][len(S['data'])-t_count:]

for i in range(0, t_count):
    
    if c[i] == 1:
        test_buy += 1.0;
    elif c[i] == 0:
        test_hold += 1.0;
    elif c[i] == -1:
        test_sell += 1.0;
    
    if p[i] == c[i]:
        correct_count += 1.0
        
        if c[i] == 1:
            correct_buy += 1.0;
        elif c[i] == 0:
            correct_hold += 1.0;
        elif c[i] == -1:
            correct_sell += 1.0;

    else:        
        if c[i] == 1:
            incorrect_buy += 1.0;
        elif c[i] == 0:
            incorrect_hold += 1.0;
        elif c[i] == -1:
            incorrect_sell += 1.0;            

print( "Accuracy:  ", (correct_count/t_count)*100.00 )

print( "Test buy:  ", test_buy  )
print( "Test sell: ", test_sell )
print( "Test hold: ", test_hold )

print( "Correct buy:  ", correct_buy,  "Incorrect buy:  ", incorrect_buy,  "Accuracy: ", (correct_buy/test_buy)*100 )
print( "Correct sell: ", correct_sell, "Incorrect sell: ", incorrect_sell, "Accuracy: ", (correct_sell/test_sell)*100 )
print( "Correct hold: ", correct_hold, "Incorrect hold: ", incorrect_hold, "Accuracy: ", (correct_hold/test_hold)*100 )


('Accuracy:  ', 77.734375)
('Test buy:  ', 62.0)
('Test sell: ', 57.0)
('Test hold: ', 140.0)
('Correct buy:  ', 46.0, 'Incorrect buy:  ', 15.0, 'Accuracy: ', 74.19354838709677)
('Correct sell: ', 40.0, 'Incorrect sell: ', 16.0, 'Accuracy: ', 70.17543859649122)
('Correct hold: ', 113.0, 'Incorrect hold: ', 26.0, 'Accuracy: ', 80.71428571428572)


In [15]:
%matplotlib qt

#colour = []
#for i in range(0, t_count):
#    if p[i] == c[i]:
#        colour.append('r')
#    else:
#        colour.append('g')

r_count = len(returns)

clf.predict(returns[0:60])

#colour = []
#for i in range(0, r_count):

plt.plot(Z['time'], Z['zigzag'], range(0, len(train)), train)
#plt.scatter(S['index'], (S['data'].T)[59], s=60, c='r')

#print(len((S['data'].T)[59]))
#print(S['data'].shape)




/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
